<a href="https://colab.research.google.com/github/Jenni01-12/Parcial-Final/blob/main/Parcial_final_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Titulo

# **Instalación de librerías**

In [ ]:
!pip install transformers datasets faiss-cpu sentence-transformers
!pip install transformers gradio --quiet
!pip install pypdf langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Cargar modelo preentrenado GPT-2
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Crear pipeline de generación de texto
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)


Device set to use cpu


In [ ]:
def responder(pregunta):
    prompt = f"Explícamelo como si fuera principiante en economía: {pregunta}\nRespuesta:"
    respuesta = chatbot(prompt, max_length=150, num_return_sequences=1, do_sample=True)[0]["generated_text"]
    return respuesta[len(prompt):].strip()


In [ ]:
import gradio as gr

interfaz = gr.Interface(fn=responder,
                        inputs="text",
                        outputs="text",
                        title="Chatbot de Economía para Dummies",
                        description="Haz cualquier pregunta de economía y obtén una respuesta sencilla")

interfaz.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5539a07f2769ab4aee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
# --- 0. Configuración Inicial ---
# Nombre del modelo Ollama a utilizar (cambia esto para usar un modelo diferente)
# Asegúrate de que este modelo exista en Ollama y sea adecuado para tareas de generación de código.
# Modelos como 'codellama', 'deepseek-coder', 'llama3' suelen ser buenos para esto.
# 'llama3' es una buena opción general.
ollama_model_name = "deepseek-r1" # Prueba con llama3, o "codellama" si quieres enfocarte en código
# ollama_model_name = "llama2:7b" # Puedes volver a llama2 si prefieres

# Nombre del archivo CSV si decides usar uno real (deja None para usar datos simulados)
# Para usar un archivo real:
# 1. Sube tu archivo CSV a tu sesión de Colab.
# 2. Cambia esta variable a "nombre_de_tu_archivo.csv"
csv_file_name = "Gravedad.csv" # Cambia a "tu_archivo.csv" para usar un archivo real

# --- 1. Instalación de Librerías y Ollama ---

print("--- Instalando librerías Python ---")
# Instalamos/actualizamos pandas, numpy, ollama y gradio
# Usamos --upgrade para asegurar versiones compatibles y -q para reducir el output
# ¡pandas y numpy ya deberían estar bien de la vez anterior, pero --upgrade no hace daño!
!pip install --upgrade pandas numpy ollama gradio openpyxl -q

# **Verificación de importaciones**
print("\n--- Verificando importaciones básicas ---")
try:
    import pandas as pd
    import numpy as np
    import ollama
    import gradio as gr
    import matplotlib.pyplot as plt # Para plots generados por código
    import io
    import sys # Importar sys para capturar stdout
    print("¡pandas, numpy, ollama, gradio, matplotlib, io y sys importados correctamente!")
except ImportError as e:
    print(f"ERROR CRÍTICO: No se pudo importar una librería necesaria: {e}")
    print("Asegúrate de que las instalaciones se completaron sin errores graves.")
    # import sys; sys.exit("Fallo en importaciones básicas.") # Descomentar si quieres detenerte aquí
except Exception as e:
    print(f"ERROR inesperado durante la importación: {e}")
    # import sys; sys.exit("Fallo en importaciones básicas.") # Descomentar si quieres detenerte aquí


# Instalar Ollama en Colab (solo es necesario ejecutar una vez por sesión)
print("\n--- Instalando Ollama ---")
!curl -fsSL https://ollama.com/install.sh | sh

# --- 2. Iniciar el servidor de Ollama en segundo plano ---
print("\n--- Iniciando servidor Ollama ---")
!pkill ollama || true # Asegura que no hay procesos ollama corriendo
!nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 & # Ejecuta en segundo plano

# --- 3. Esperar a que el servidor de Ollama inicie completamente ---
import time
print("Esperando a que el servidor Ollama esté listo (15 segundos)...")
time.sleep(15) # Ajusta este tiempo si ves errores de conexión iniciales

# --- 4. Verificar que el servidor esté respondiendo ---
print("\n--- Verificando servidor Ollama ---")
!curl -s http://localhost:11434/api/tags || echo "El servidor Ollama no está respondiendo. Revisa los logs o aumenta el tiempo de espera."

# --- 5. Descargar el modelo especificado ---
print(f"\n--- Descargando modelo {ollama_model_name} desde Ollama ---")
# Si cambiaste ollama_model_name, esto descargará el nuevo. Si ya lo tienes, será rápido.
!ollama pull {ollama_model_name}
print(f"Modelo {ollama_model_name} descargado (o ya disponible).")

# --- 6. Preparar los datos (Simulados o cargados desde CSV) ---
# pandas y os ya deberían estar importados desde la verificación inicial
import os

print("\n--- Cargando datos ---")

df = None # Inicializar el dataframe a None

if csv_file_name and os.path.exists(csv_file_name):
    # Si se especificó un archivo CSV y existe, cargarlo
    try:
        df = pd.read_csv(csv_file_name)
        print(f"Datos cargados desde '{csv_file_name}'. Filas: {len(df)}, Columnas: {len(df.columns)}")
    except Exception as e:
        print(f"Error al cargar el archivo CSV '{csv_file_name}': {e}")
        print("Se procederá a usar datos simulados.")
        csv_file_name = None # Revertir para usar datos simulados si falla la carga
else:
    if csv_file_name:
        print(f"Archivo '{csv_file_name}' no encontrado.")
        print("Se procederá a usar datos simulados.")
        csv_file_name = None # Revertir para usar datos simulados si no se encuentra


--- Instalando librerías Python ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [5]:
print("\nPrimeras 5 filas del DataFrame:")
print(df.head().to_markdown(index=False)) # Mostrar como markdown para mejor legibilidad en Colab
print(f"\nColumnas y tipos de datos:")
display(df) # Usar df.info() directamente para mostrar en la consola de Colab
# --- 7. Funciones para interactuar con Ollama y ejecutar código ---

# Pre-generar la descripción del DataFrame que se enviará al LLM
# Esto se hace una vez después de cargar o crear el DataFrame
# Usamos io.StringIO() para capturar el output de df.info() como string para el LLM
buffer = io.StringIO()
df.info(verbose=True, buf=buffer)
dataframe_description = f"""Tienes acceso a un pandas DataFrame llamado 'df'.
Su información (columnas, tipos, no-nulos) es:
{buffer.getvalue()}

Primeras filas:
{df.head().to_string()}
"""



Primeras 5 filas del DataFrame:
|   Año | Importador   | Exportador   |          Exports |   Distancia |   PIBorigen |   PIBdestino |   Arancel |   TLC |   Frontera |   Relaciones |   Religión |   Colonia |
|------:|:-------------|:-------------|-----------------:|------------:|------------:|-------------:|----------:|------:|-----------:|-------------:|-----------:|----------:|
|  2021 | Angola       | Albania      | 212509           |     5682.57 |  1.8032e+10 |  6.65051e+10 | 0.1325    |     0 |          0 |            0 |    0       |         0 |
|  2021 | Argentina    | Albania      |      1.0933e+06  |    11628    |  1.8032e+10 |  4.86564e+11 | 0.178533  |     0 |          0 |            0 |    0.00041 |         0 |
|  2021 | Armenia      | Albania      |      1.59539e+06 |     2082    |  1.8032e+10 |  1.38789e+10 | 0.0217263 |     0 |          0 |            0 |    0       |         0 |
|  2021 | Australia    | Albania      |      3.95907e+06 |    15120.7  |  1.8032e+10 |  1.55

,Año,Importador,Exportador,Exports,Distancia,PIBorigen,PIBdestino,Arancel,TLC,Frontera,Relaciones,Religión,Colonia
0,2021,Angola,Albania,2.125090e+05,5682.5713,1.803201e+10,6.650513e+10,0.132500,0.0,0,0,0.000000,0
1,2021,Argentina,Albania,1.093298e+06,11628.0000,1.803201e+10,4.865640e+11,0.178533,0.0,0,0,0.000410,0
2,2021,Armenia,Albania,1.595392e+06,2082.0000,1.803201e+10,1.387891e+10,0.021726,0.0,0,0,0.000000,0
3,2021,Australia,Albania,3.959069e+06,15120.6610,1.803201e+10,1.556740e+12,0.000000,0.0,0,0,0.000410,0
4,2021,Austria,Albania,5.043321e+07,811.0000,1.803201e+10,4.804670e+11,0.057346,1.0,0,0,0.001230,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55554,2023,Ukraine,Zimbabwe,7.054224e+06,7436.9229,3.523137e+10,1.787570e+11,0.027358,0.0,0,0,0.000000,0
55555,2023,United Arab Emirates,Zimbabwe,4.068426e+09,5504.0684,3.523137e+10,5.141300e+11,0.065000,0.0,0,0,0.009759,1
55556,2023,United Kingdom,Zimbabwe,2.116518e+07,8466.8115,3.523137e+10,3.380850e+12,0.025576,1.0,0,0,0.053444,0
55557,2023,United States of America,Zimbabwe,1.186300e+08,13976.4760,3.523137e+10,2.772070e+13,0.036200,0.0,0,1,0.136576,0


In [6]:
def generate_python_code(prompt, dataframe_info=dataframe_description, model=ollama_model_name):
    """
    Envía un prompt a Ollama pidiendo generar código Python para analizar el DataFrame.
    Devuelve solo el código Python generado.
    """
    # Este es el prompt clave para dirigir al LLM
        system_prompt = f"""Eres un analista económico especializado en comercio internacional y modelos de gravedad.
Tu tarea es generar Gráficos profesionales y análisis de los gráficos extensivos, resumen y pronósticos teniendo en cuenta el DataFrame 'df'.
    El DataFrame tiene la siguiente estructura y tipos de datos:
    {dataframe_info}

    Reglas estrictas para tu respuesta:
    1.  NO DEVUELVE CODIGO EN PYTHON. Se centra en la visualización de datos y el texto
    2. Prohibido incluir explicaciones en texto libre
    3. No usar bloques python  o >>> prompts
    4. Usar gráficos claros (barras horizontales > verticales)
    5. Títulos descriptivos con unidades (USD, km, %)
    6. Escalas logarítmicas cuando haya gran dispersión
    7. Última línea DEBE ser el pronóstico o el análsis de la gráfica en caso de ser necesitado. Si no, solo la gráfica
    Crear variable markdown_summary con:
    8. Título con conclusión principal
    9. 3-4 bullet points de hallazgos
    10. Factores económicos relevantes
    11. Formato compatible con PowerPoint
    PROHIBIDO:
    12. Mostrar DataFrames >10 filas sin filtrar
    13. Gráficos sin etiquetas de ejes
    14. Análisis sin contexto económico
    15. Términos técnicos sin explicar


    """

    user_prompt = f"Pregunta del usuario: {prompt}"

    print(f"\n--- Enviando prompt a {model} para generar código ---")
    # Usamos la librería ollama para interactuar con el servidor local
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt},
            ],
            options={
                "temperature": 0.2, # Mantener bajo para código preciso
                # "num_predict": 512 # Opcional: limitar longitud si es necesario
            }
        )
        # Asegurarse de que solo se extrae el contenido de la respuesta del modelo
        code = response['message']['content'].strip()

        # A veces, los LLMs pueden incluir markdown a pesar de las instrucciones estrictas.
        # Intentar limpiar bloques de código markdown si aparecen.
        if code.startswith("```python"):
            code = code.replace("```python", "").strip()
            if code.endswith("```"):
                code = code[:-3].strip()
        elif code.startswith("```"): # Otros bloques de código
             code = code.replace("```", "").strip()


        print("Código Python generado:")
        print("--- INICIO CÓDIGO ---")
        print(code)
        print("--- FIN CÓDIGO ---")
        return code
    except Exception as e:
        print(f"Error al llamar a Ollama para generar código: {e}")
        return None
